# Omnipose Segmentation from ImageJ Macro converted image directories

This file is meant to aid in omnipose segmentation in a reproducible and streamlined way to help with automated image analysis especially early QC to adjust experimental and imaging parameters as needed to optimize S/N for the experiment. 

#### Import Necessary packages and Functions

In [1]:
# Imports for all chunks
import os
import shutil
from aicsimageio.readers.ome_tiff_reader import OmeTiffReader
import numpy as np
from cellpose_omni import models, core
import matplotlib as mpl
import matplotlib.pyplot as plt
from skimage.io import imread, imsave
from pathlib import Path
import time


c:\Users\MicrobeJ\anaconda3\envs\omnipose_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:


import os
import shutil
from aicsimageio.readers.ome_tiff_reader import OmeTiffReader

# Mapping dictionary for renaming channels
channel_map = {'Phase': 'phase', 'eGFP': 'fish', 'DAPI': 'dapi'}

# Root directory
root_dir = 'C:\\Users\\MicrobeJ\\Downloads\\omnipose_multichannel\\sandbox_2'

# Navigate through directories to find OME.TIFF files and rename them
for biorep_dir in os.listdir(root_dir):
    biorep_path = os.path.join(root_dir, biorep_dir)
    if os.path.isdir(biorep_path):
        for date_strain_dir in os.listdir(biorep_path):
            date_strain_path = os.path.join(biorep_path, date_strain_dir)
            if os.path.isdir(date_strain_path):
                for sub_dir in os.listdir(date_strain_path):
                    sub_dir_path = os.path.join(date_strain_path, sub_dir)
                    if os.path.isdir(sub_dir_path):
                        for img_data_dir in os.listdir(sub_dir_path):
                            img_data_path = os.path.join(sub_dir_path, img_data_dir)
                            if os.path.isdir(img_data_path):
                                for file in os.listdir(img_data_path):
                                    if file.endswith('.ome.tiff') or file.endswith('.ome.tif'):
                                        file_path = os.path.join(img_data_path, file)
                                        
                                        # Read the OME.TIFF file to get channel names
                                        reader = OmeTiffReader(file_path)
                                        ome_metadata = reader.ome_metadata
                                        channel_names = [channel.name for channel in ome_metadata.images[0].pixels.channels]
                                        
                                        # Rename folders and files based on channel names
                                        for i, channel_name in enumerate(channel_names):
                                            # Map the original channel name to the new name using the channel_map dictionary
                                            mapped_name = channel_map.get(channel_name, channel_name)
                                            
                                            # Create the old and new folder names based on channel index
                                            old_folder_name = f"C{i+1}-MAX_sequence"
                                            new_folder_name = f"{mapped_name}-MAX_sequence"
                                            
                                            # Create the full path to the old and new folder names
                                            old_folder_path = os.path.join(img_data_path, old_folder_name)
                                            new_folder_path = os.path.join(img_data_path, new_folder_name)
                                            
                                            # If the old folder exists, rename it to the new folder name
                                            if os.path.exists(old_folder_path):
                                                shutil.move(old_folder_path, new_folder_path)
                                            
                                            # Rename individual single-page TIFF files inside the new folder
                                            for single_tiff in os.listdir(new_folder_path):
                                                # Check if the file starts with the old channel name
                                                if single_tiff.startswith(f"C{i+1}-MAX"):
                                                    # Create the full path to the old single-page TIFF file
                                                    old_single_tiff_path = os.path.join(new_folder_path, single_tiff)
                                                    
                                                    # Create the new single-page TIFF file name based on mapped channel name
                                                    new_single_tiff_name = single_tiff.replace(f"C{i+1}-MAX", f"{mapped_name}-MAX")
                                                    
                                                    # Create the full path to the new single-page TIFF file
                                                    new_single_tiff_path = os.path.join(new_folder_path, new_single_tiff_name)
                                                    
                                                    # Rename the old single-page TIFF file to the new name
                                                    shutil.move(old_single_tiff_path, new_single_tiff_path)
                                            
                                            # Create old and new multi-page TIFF file names based on channel index
                                            old_file_name = f"C{i+1}-MAX.tif"
                                            new_file_name = f"{mapped_name}-MAX.tif"
                                            
                                            # Create the full path to the old and new multi-page TIFF files
                                            old_file_path = os.path.join(img_data_path, old_file_name)
                                            new_file_path = os.path.join(img_data_path, new_file_name)
                                            
                                            # If the old multi-page TIFF file exists, rename it to the new name
                                            if os.path.exists(old_file_path):
                                                shutil.move(old_file_path, new_file_path)




## Running Omnipose for Segmentation

Here is the incorporation into the omnipose script

### Importing the images and QC to check images match expectations

### Collecting all the tiff files for omnipose


In [5]:
from skimage import io  # Importing the io module from skimage for image reading

# Initialize an empty list to store the full paths of all phase-MAX_sequence TIFF files.
# This list will include both newly renamed and previously renamed phase files.
all_phase_max_sequence_files = []

# Counter for total images
total_images = 0

# Counter for images with issues
issues_counter = 0

# Use os.walk to navigate through the directory tree rooted at root_dir.
# os.walk yields a 3-tuple (dirpath, dirnames, filenames) for each directory it visits.
# dirpath is the path to the current directory, dirnames is a list of subdirectories in the current directory,
# and filenames is a list of filenames in the current directory.

# Loop through the directory structure
for root, dirs, files in os.walk(root_dir):
    for dir in dirs:
        if dir == "phase-MAX_sequence":
            phase_folder_path = os.path.join(root, dir)
            for file in os.listdir(phase_folder_path):
                if file.endswith(".tif"):
                    full_file_path = os.path.join(phase_folder_path, file)
                    all_phase_max_sequence_files.append(full_file_path)
                    
                    # Read the image into an array
                    img = io.imread(full_file_path)
                    
                    # Perform quality checks
                    shape = img.shape
                    dtype = img.dtype
                    min_val, max_val = img.min(), img.max()

                    # Increment the total_images counter
                    total_images += 1

                    #quality control checks here
                    if shape != (512, 512) or min_val < 3500 or max_val > 28000:
                        issues_counter += 1
                        print(f"Warning: Image at {full_file_path} has issues.")
                        print(f"  - Original image shape: {shape}")
                        print(f"  - Data type: {dtype}")
                        print(f"  - Data range: min {min_val}, max {max_val}")

print(f"\nTotal number of images processed: {total_images}")
if issues_counter:
    print(f"Number of images with issues: {issues_counter}")
else:
    print("No issues found in images.")



Total number of images processed: 56
No issues found in images.


In [17]:
import os
import numpy as np
from skimage.io import imread, imsave

# Initialize model
model_name = 'bact_phase_omni'
model = models.CellposeModel(gpu=False, model_type=model_name)


# define parameters
params = {
    'channels': [0,0],  # Segment based on first channel, no second channel
    'rescale': None,  # upscale or downscale your images, None = no rescaling
    'mask_threshold': -1,  # erode or dilate masks with higher or lower values
    'flow_threshold': 0,  # default is .4, but only needed if there are spurious masks to clean up; slows down output
    'transparency': True,  # transparency in flow output
    'omni': True,  # we can turn off Omnipose mask reconstruction, not advised
    'cluster': True,  # use DBSCAN clustering
    'resample': True,  # whether or not to run dynamics on rescaled grid or original grid
    'verbose': False,  # turn on if you want to see more output
    'tile': False,  # average the outputs from flipped (augmented) images; slower, usually not needed
    'niter': None,  # None lets Omnipose calculate # of Euler iterations (usually <20) but you can tune it for over/under segmentation
    'augment': False,  # Can optionally rotate the image and average outputs, usually not needed
    'affinity_seg': False,  # new feature, stay tuned...
}


# Starting directory (Replace with your actual directory)
start_directory = "sandbox_2/biorep1/20230801_LZ22225_imaging/Image_Data/"

# First loop: Generate masks for 'phase-MAX_sequence' in each experiment directory
for experiment in os.listdir(start_directory):
    experiment_path = os.path.join(start_directory, experiment)
    
    if os.path.isdir(experiment_path):
        channel_sequence_path = os.path.join(experiment_path, 'phase-MAX_sequence')
        
        if os.path.exists(channel_sequence_path):
            all_images = [f for f in os.listdir(channel_sequence_path) if f.endswith('.tif')]
            
            for img_file in all_images:
                img_path = os.path.join(channel_sequence_path, img_file)
                
                # Read the image and generate the mask
                image = imread(img_path)
                masks, _, _ = model.eval(image, **params)
                
                # Save the mask
                mask_path = os.path.join(channel_sequence_path, img_file.replace('.tif', '_mask.tif'))
                imsave(mask_path, masks.astype(np.uint16))

# Sanity check for the size of the masks
total_size_MB = np.array(masks_list).nbytes / (1024 * 1024)  # size in MB
print(f"Total size of stored masks: {total_size_MB:.2f} MB")




2023-10-17 16:26:04,567 [INFO] >>bact_phase_omni<< model set to be used
2023-10-17 16:26:04,569 [INFO] >>>> using CPU
Total size of stored masks: 28.00 MB


### Segmentation


In [16]:
from skimage.io import imread, imsave
from skimage import img_as_uint 
import numpy as np
from cellpose_omni import models, utils, io as cellpose_io
from skimage.measure import label, regionprops
from skimage.color import label2rgb
import time

# Check for CUDA-enabled GPU availability
# Uncomment this block when you want to switch to GPU computation
'''
import torch

# Check for GPU availability and set the gpu flag
if torch.cuda.is_available():
    gpu = True
    print("CUDA-enabled GPU found. Switching to GPU mode.")
else:
    gpu = False
    print("No CUDA-enabled GPU found. Running on CPU.")
'''

# The `gpu` variable can then be passed to the CellposeModel initialization like so:
# model = models.CellposeModel(gpu=gpu, model_type=model_name)




# Initialize model
model_name = 'bact_phase_omni'
model = models.CellposeModel(gpu=False, model_type=model_name)


# define parameters
params = {
    'channels': [0,0],  # Segment based on first channel, no second channel
    'rescale': None,  # upscale or downscale your images, None = no rescaling
    'mask_threshold': -1,  # erode or dilate masks with higher or lower values
    'flow_threshold': 0,  # default is .4, but only needed if there are spurious masks to clean up; slows down output
    'transparency': True,  # transparency in flow output
    'omni': True,  # we can turn off Omnipose mask reconstruction, not advised
    'cluster': True,  # use DBSCAN clustering
    'resample': True,  # whether or not to run dynamics on rescaled grid or original grid
    'verbose': False,  # turn on if you want to see more output
    'tile': False,  # average the outputs from flipped (augmented) images; slower, usually not needed
    'niter': None,  # None lets Omnipose calculate # of Euler iterations (usually <20) but you can tune it for over/under segmentation
    'augment': False,  # Can optionally rotate the image and average outputs, usually not needed
    'affinity_seg': False,  # new feature, stay tuned...
}



## Segmentation and post-processing
for file in all_phase_max_sequence_files:
    # Read the image
    image = imread(file)
    
    # Apply the model
    masks, flows, styles = model.eval(image, **params)
    
    # Post-process
    label_image = label(masks)
    
    # Generate cell-only and background-only images
    cell_only_image = image * (masks > 0)
    background_only_image = image * (masks == 0)
    
    # Generate paths
    directory = os.path.dirname(file)
    filename = os.path.basename(file)
    base_name = os.path.splitext(filename)[0]
    
    output_cell_only_path = os.path.join(directory, f"{base_name}_cell_only.tif")
    output_background_only_path = os.path.join(directory, f"{base_name}_background_only.tif")
    output_outlines_path = os.path.join(directory, f"{base_name}_outlines.txt")
    output_mask_path = os.path.join(directory, f"{base_name}_mask.tif")
    
    # Save the images and outlines
    imsave(output_cell_only_path, img_as_uint(cell_only_image))
    imsave(output_background_only_path, img_as_uint(background_only_image))
    cellpose_io.outlines_to_text(output_outlines_path, label_image)
    imsave(output_mask_path, masks.astype(np.uint16))



2023-10-17 16:06:14,312 [INFO] >>bact_phase_omni<< model set to be used
2023-10-17 16:06:14,359 [INFO] >>>> using CPU


C:\Users\MicrobeJ\AppData\Local\Temp\ipykernel_32944\311679488.py:82: UserWarning: C:\Users\MicrobeJ\Downloads\omnipose_multichannel\sandbox_2\biorep1\20230801_LZ22225_imaging\Image_Data\2023.08.01_LZ22225_10min_uninf.004\phase-MAX_sequence\phase-MAX-0001_mask.tif is a low contrast image
  imsave(output_mask_path, masks.astype(np.uint16))
C:\Users\MicrobeJ\AppData\Local\Temp\ipykernel_32944\311679488.py:82: UserWarning: C:\Users\MicrobeJ\Downloads\omnipose_multichannel\sandbox_2\biorep1\20230801_LZ22225_imaging\Image_Data\2023.08.01_LZ22225_10min_uninf.004\phase-MAX_sequence\phase-MAX-0002_mask.tif is a low contrast image
  imsave(output_mask_path, masks.astype(np.uint16))
C:\Users\MicrobeJ\AppData\Local\Temp\ipykernel_32944\311679488.py:82: UserWarning: C:\Users\MicrobeJ\Downloads\omnipose_multichannel\sandbox_2\biorep1\20230801_LZ22225_imaging\Image_Data\2023.08.01_LZ22225_10min_uninf.004\phase-MAX_sequence\phase-MAX-0003_mask.tif is a low contrast image
  imsave(output_mask_path, ma